# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import scipy.stats as st
import json


# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


In [2]:
from ipywidgets import interact
@interact(x=(0, 100, 10))
def p(x=50):
    pass



interactive(children=(IntSlider(value=50, description='x', step=10), Output()), _dom_classes=('widget-interact…

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
clean_data_path='../Weather Py/clean_city_data.csv'
wpy_df=pd.read_csv(clean_data_path)
wpy_df.head()

,Unnamed: 0,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country
0,0,Ushuaia,-68.30,-54.80,46.40,4.70,75.0,20.0,AR
1,0,Punta Arenas,-70.92,-53.15,41.00,6.93,86.0,100.0,CL
2,0,Bluff,168.33,-46.60,57.29,7.85,73.0,0.0,NZ
3,0,Kaitangata,169.85,-46.28,60.01,4.00,60.0,0.0,NZ
4,0,Mataura,168.86,-46.19,59.83,3.47,62.0,0.0,NZ


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configuring gmaps
#Grab lat and lon from wpy_df to use in gmaps.  Grab humidity data for location value
wpy_df['lat_lon']=""
lat_lon=wpy_df[['Lat','Lon']]
humidity=wpy_df['Humidity']
lat_lon

,Lat,Lon
0,-54.80,-68.30
1,-53.15,-70.92
2,-46.60,168.33
3,-46.28,169.85
4,-46.19,168.86
...,...,...
542,71.97,102.50
543,72.79,-56.15
544,73.51,80.55
545,77.48,-69.36


In [6]:
#Create google map figure to place heat map
#gmaps.configure(api_key=gkey)
hum_fig=gmaps.figure(zoom_level=2.0,center=(0, 0) )

hum_heat=gmaps.heatmap_layer(lat_lon)

heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius=1)

#Add heatmap to figure
hum_fig.add_layer(heat_layer)
hum_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Already dropped null values.  Used criteria to select ideal weather and filter df for those locations
wpy_df
nice_df=wpy_df[(wpy_df['Humidity']>40) & (wpy_df['Humidity']<60)]
nice_df=nice_df[(nice_df['Max Temp']>65) &(nice_df['Max Temp']<85)]
nice_df=nice_df[nice_df['Cloud Cover']<50]
nice_df=nice_df[nice_df['Wind Speed']<10]
#nice_df=nice_df.dropna(axis=0, how="any")
nice_df

,Unnamed: 0,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon
72,0,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,
73,0,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,
77,0,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,
115,0,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,
212,0,Yuli,10.27,9.70,84.79,6.62,54.0,4.0,NG,
235,0,Nioro,-15.75,13.35,76.82,5.70,43.0,39.0,GM,
252,0,Chittarkonda,82.08,18.12,81.88,1.92,55.0,0.0,IN,
261,0,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,
263,0,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,
269,0,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Remove unnamed column.  Create new column for Hotel Name
hotel_df=nice_df
hotel_df=hotel_df.drop(['Unnamed: 0'], axis=1)
hotel_df['Hotel Name']=''
hotel_df

,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,,
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,,
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,,
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,,
212,Yuli,10.27,9.70,84.79,6.62,54.0,4.0,NG,,
235,Nioro,-15.75,13.35,76.82,5.70,43.0,39.0,GM,,
252,Chittarkonda,82.08,18.12,81.88,1.92,55.0,0.0,IN,,
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,,
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,,
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,,


In [10]:
#view hotel_df
hotel_df


,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,,
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,,
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,,
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,,
212,Yuli,10.27,9.70,84.79,6.62,54.0,4.0,NG,,
235,Nioro,-15.75,13.35,76.82,5.70,43.0,39.0,GM,,
252,Chittarkonda,82.08,18.12,81.88,1.92,55.0,0.0,IN,,
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,,
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,,
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,,


In [12]:
#target_coordinates = (str(hotel_df['Lat'])+", "+str(hotel_df['Lon']))
#print(target_coordinates)
#Creating new lat_lon column to pass into json request for latitidue and longitude.  Could also use variable=hotel_df[['Lat', 'Lon']] I believe.
hotel_df['lat_lon']=""
xy=[]
for index, row in hotel_df.iterrows():
    x=str(row['Lat']) + ', ' + str(row['Lon'])
    xy.append(x)
hotel_df['lat_lon']=xy
hotel_df


,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,"-22.97, -42.03",
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,"-22.96, 14.51",
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,"-22.12, 14.28",
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,"-15.2, 12.15",
212,Yuli,10.27,9.70,84.79,6.62,54.0,4.0,NG,"9.7, 10.27",
235,Nioro,-15.75,13.35,76.82,5.70,43.0,39.0,GM,"13.35, -15.75",
252,Chittarkonda,82.08,18.12,81.88,1.92,55.0,0.0,IN,"18.12, 82.08",
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,"19.67, 97.21",
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,"19.93, -105.25",
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,"21.32, -158.01",


In [16]:
# set up a parameters dictionary, base url to search, and variable lists to fill
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"
hotel_name=[]
country_name=[]
# set up a parameters dictionary
p_dict = {    
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# use iterrows to iterate through hotel dataframe
for index, row in hotel_df.iterrows():

# get location from lat_lon
# Help from Hunter Carlisle on try and exception section in for-loop below.  Used to skip sections where there is no response within parameters.
# Found city of Yuli had no hotel within 5000 meters based on lat, lon
    try:
        location = row['lat_lon']


    # add keyword to params dict
        p_dict['location'] = location

    # assemble url and make API request
        response = requests.get(base_url, params=p_dict).json()

    # extract results
        results = response['results']
        #print results name.  Used this in my troubleshooting
        print(results[0]['name'])
        hotel=results[0]['name']
        hotel_name.append(hotel)
    except IndexError as error:
        hotel_name.append("")

hotel_df['Hotel Name']=hotel_name

hotel_df

Hotel da Canoa
Protea Hotel by Marriott Walvis Bay Pelican Bay
De Duine Hotel
Hotel ibis Styles Iu Namibe
Gold Hotel
Hotel Hacienda Vieja
Hampton Inn & Suites Oahu/Kapolei
Hampton Inn & Suites Oahu/Kapolei
Ratnodweep Resort
Risk Hotel
Al Najada Doha Hotel by Tivoli
Depche Village Resort


,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,"-22.97, -42.03",Hotel da Canoa
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,"-22.96, 14.51",Protea Hotel by Marriott Walvis Bay Pelican Bay
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,"-22.12, 14.28",De Duine Hotel
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,"-15.2, 12.15",Hotel ibis Styles Iu Namibe
212,Yuli,10.27,9.70,84.79,6.62,54.0,4.0,NG,"9.7, 10.27",
235,Nioro,-15.75,13.35,76.82,5.70,43.0,39.0,GM,"13.35, -15.75",
252,Chittarkonda,82.08,18.12,81.88,1.92,55.0,0.0,IN,"18.12, 82.08",
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,"19.67, 97.21",Gold Hotel
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,"19.93, -105.25",Hotel Hacienda Vieja
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,"21.32, -158.01",Hampton Inn & Suites Oahu/Kapolei


In [17]:
#Remove hotel name = "".  This is byproduct of try exception statement above.  Rather than use expection: pass, replaced those cells with "" in order to keep order in loop.
hotel_df_cleaned = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df_cleaned

,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,"-22.97, -42.03",Hotel da Canoa
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,"-22.96, 14.51",Protea Hotel by Marriott Walvis Bay Pelican Bay
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,"-22.12, 14.28",De Duine Hotel
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,"-15.2, 12.15",Hotel ibis Styles Iu Namibe
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,"19.67, 97.21",Gold Hotel
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,"19.93, -105.25",Hotel Hacienda Vieja
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,"21.32, -158.01",Hampton Inn & Suites Oahu/Kapolei
271,Makakilo City,-158.09,21.35,84.20,9.17,45.0,1.0,US,"21.35, -158.09",Hampton Inn & Suites Oahu/Kapolei
280,Pābna,89.25,24.00,83.53,5.73,55.0,0.0,BD,"24.0, 89.25",Ratnodweep Resort
281,Darlawn,92.90,24.02,70.48,2.19,52.0,0.0,IN,"24.02, 92.9",Risk Hotel


In [20]:
#Create new column called City that is equivalent to the name column since that is what info box template coding appears to be requesting below.
hotel_df_cleaned['City'] = hotel_df_cleaned['name']

C:\Users\Pete\anaconda3\envs\gmaps\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
#View hotel_df_cleaned
hotel_df_cleaned

,name,Lon,Lat,Max Temp,Wind Speed,Humidity,Cloud Cover,Country,lat_lon,Hotel Name,City
72,Arraial do Cabo,-42.03,-22.97,71.40,6.64,57.0,46.0,BR,"-22.97, -42.03",Hotel da Canoa,Arraial do Cabo
73,Walvis Bay,14.51,-22.96,70.41,5.50,44.0,0.0,NaN,"-22.96, 14.51",Protea Hotel by Marriott Walvis Bay Pelican Bay,Walvis Bay
77,Henties Bay,14.28,-22.12,68.38,7.43,52.0,0.0,NaN,"-22.12, 14.28",De Duine Hotel,Henties Bay
115,Mossamedes,12.15,-15.20,78.71,2.68,55.0,0.0,AO,"-15.2, 12.15",Hotel ibis Styles Iu Namibe,Mossamedes
261,Loikaw,97.21,19.67,80.98,0.98,52.0,0.0,MM,"19.67, 97.21",Gold Hotel,Loikaw
263,Tomatlán,-105.25,19.93,84.69,6.80,49.0,0.0,MX,"19.93, -105.25",Hotel Hacienda Vieja,Tomatlán
269,‘Ewa Beach,-158.01,21.32,84.20,9.17,45.0,1.0,US,"21.32, -158.01",Hampton Inn & Suites Oahu/Kapolei,‘Ewa Beach
271,Makakilo City,-158.09,21.35,84.20,9.17,45.0,1.0,US,"21.35, -158.09",Hampton Inn & Suites Oahu/Kapolei,Makakilo City
280,Pābna,89.25,24.00,83.53,5.73,55.0,0.0,BD,"24.0, 89.25",Ratnodweep Resort,Pābna
281,Darlawn,92.90,24.02,70.48,2.19,52.0,0.0,IN,"24.02, 92.9",Risk Hotel,Darlawn


In [26]:
#create hotel_info list to store data
hotel_info=[]
#create list of variables to run through for-loop in cell below
locations=hotel_df_cleaned[['Lat', 'Lon']]





In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_cleaned.iterrows()]
locations = hotel_df_cleaned[["Lat", "Lon"]]

In [24]:
#Create new gmap marker_layer
new_layer=gmaps.marker_layer(locations, label='Click for more Info', info_box_content=hotel_info)

In [25]:
# Add marker layer ontop of humidity heat map
hum_fig.add_layer(new_layer)

# Display figure
hum_fig

Figure(layout=FigureLayout(height='420px'))